In [ ]:
import polars as pl
from datetime import date

### Данные

In [17]:
stay_date_expr = (
    pl.concat_str(
        [
            pl.col("year").cast(pl.Utf8),
            pl.lit("-"),
            pl.col("month").cast(pl.Utf8).str.zfill(2),
            pl.lit("-"),
            pl.col("date").cast(pl.Utf8).str.zfill(2),
        ]
    )
    .str.strptime(pl.Date, "%Y-%m-%d", strict=False)
)


In [18]:
logs_hotel_lz = (
    pl.scan_csv("data/Hotel.csv", try_parse_dates=True)
    .with_columns(
        (pl.col("weekend_nights") + pl.col("week_nights")).alias("total_nights"),
        (pl.col("n_adults") + pl.col("n_children")).alias("total_guests"),
        stay_date_expr.alias("stay_date"),
        pl.col("status").str.to_lowercase().alias("status"),
    )
)


### Calendar dimension

In [ ]:
calendar_dt = pl.DataFrame(
    {
        "calendar_dt": pl.date_range(
            date(2017, 1, 1),
            date(2018, 12, 31),
            interval="1d",
            eager=True,
        )
    }
)


In [ ]:
def run(lazy_expr, label):
    print(f"───────────────────────────────────── {label} ─────────────────────────────────────")
    print(lazy_expr.explain())
    return lazy_expr.collect()

### Запросы

In [ ]:
# Среднее кол-во ночей
q_avg_nights_lz = logs_hotel_lz.select(pl.col("total_nights").mean().alias("avg_nights"))

# b) Кол-во отмен, %
q_cancel_rate_lz = (
    logs_hotel_lz
    .group_by("year", "month")
    .agg(
        (pl.col("status") == "canceled").sum().alias("canceled"), 
        pl.count().alias("total"),
    )
    .with_columns((pl.col("canceled") / pl.col("total") * 100).round(2).alias("cancel_pct"))  
    .select("year", "month", "cancel_pct")
)

# c) Выручка за каждый месяц по типу бронирования
q_revenue_lz = (
    logs_hotel_lz
    .filter(pl.col("status") != "canceled")
    .with_columns((pl.col("avg_room_price") * pl.col("total_nights")).alias("booking_revenue"))
    .group_by("year", "month", "market_segment")
    .agg(pl.col("booking_revenue").sum().alias("revenue"))
    .sort(["year", "month", "market_segment"])
)

# d) Среднее время между бр-м и заездом
q_leadtime_lz = (
    logs_hotel_lz
    .group_by("year", "month")
    .agg(pl.col("lead_time").mean().alias("avg_lead_time"))
    .sort(["year", "month"])
)

# e) Кол-во гостей в день
q_guests_day_lz = (
    logs_hotel_lz
    .group_by("stay_date")
    .agg(pl.col("total_guests").sum().alias("guests"))  
    .sort("guests", descending=True)
)

In [30]:
if __name__ == "__main__": 
    results = {
        "Среднее кол-во ночей": run(q_avg_nights_lz, "a) Среднее кол-во ночей"),
        "Кол-во отмен, %": run(q_cancel_rate_lz, "b) Кол-во отмен, %"),
        "Выручка за каждый месяц по типу бронирования": run(q_revenue_lz, 
                                                            "c) Выручка за каждый месяц по типу бронирования"),
        "Среднее время между бр-м и заездом": run(q_leadtime_lz, "d) Среднее время между бр-м и заездом"),
        "Кол-во гостей в день": run(q_guests_day_lz, "e) Кол-во гостей в день"),
    }

───────────────────────────────────── a) Среднее кол-во ночей ─────────────────────────────────────
SELECT [col("total_nights").mean().alias("avg_nights")]
FROM
   WITH_COLUMNS:
   [[(col("weekend_nights")) + (col("week_nights"))].alias("total_nights")] 
    Csv SCAN [data/Hotel.csv]
    PROJECT 2/19 COLUMNS
───────────────────────────────────── b) Кол-во отмен, % ─────────────────────────────────────
simple π 3/5 ["year", "month", "cancel_pct"]
   WITH_COLUMNS:
   [[([(col("canceled")) / (col("total"))]) * (100.0)].round().alias("cancel_pct")] 
    AGGREGATE
      [[(col("status")) == ("canceled")].sum().alias("canceled"), len().alias("total")] BY [col("year"), col("month")]
      FROM
       WITH_COLUMNS:
       [col("status").str.lowercase().alias("status")] 
        Csv SCAN [data/Hotel.csv]
        PROJECT 3/19 COLUMNS
───────────────────────────────────── c) Выручка за каждый месяц по типу бронирования ─────────────────────────────────────
SORT BY [col("year"), col("month"), col(

In [ ]:
for label, df in results.items():
    print(f"\n{label}\n{df}")


Среднее кол-во ночей
shape: (1, 1)
┌────────────┐
│ avg_nights │
│ ---        │
│ f64        │
╞════════════╡
│ 3.015024   │
└────────────┘

Кол-во отмен, %
shape: (18, 3)
┌──────┬───────┬────────────┐
│ year ┆ month ┆ cancel_pct │
│ ---  ┆ ---   ┆ ---        │
│ i64  ┆ i64   ┆ f64        │
╞══════╪═══════╪════════════╡
│ 2018 ┆ 12    ┆ 18.16      │
│ 2018 ┆ 7     ┆ 41.89      │
│ 2018 ┆ 5     ┆ 36.49      │
│ 2018 ┆ 11    ┆ 36.35      │
│ 2017 ┆ 7     ┆ 66.94      │
│ …    ┆ …     ┆ …          │
│ 2018 ┆ 1     ┆ 2.37       │
│ 2018 ┆ 4     ┆ 36.37      │
│ 2018 ┆ 2     ┆ 25.23      │
│ 2018 ┆ 6     ┆ 40.31      │
│ 2018 ┆ 8     ┆ 46.55      │
└──────┴───────┴────────────┘

Выручка за каждый месяц по типу бронирования
shape: (81, 4)
┌──────┬───────┬────────────────┬───────────┐
│ year ┆ month ┆ market_segment ┆ revenue   │
│ ---  ┆ ---   ┆ ---            ┆ ---       │
│ i64  ┆ i64   ┆ str            ┆ f64       │
╞══════╪═══════╪════════════════╪═══════════╡
│ 2017 ┆ 7     ┆ Complemen